In [1]:
#Creators -
#Shreejit Gajanan Deshmukh
#Venkata Sai Advaith Kandiraju
#PRCV Spring 2023' Assignment 5 - Deep learning

In [1]:
#Summary -
#This program is meant for analyzing effects on accuracy when we are changing certain aspects of our neural network

In [2]:
#Importing libraries
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
#Defining neural networks parameters
#Reruns
n_epochs = 1

#Batch size for training and testing
batch_size_train = 64
batch_size_test = 1000

#Hyperparameters
learning_rate = 0.01
momentum = 0.5

log_interval = 10

#Initializing the values we are changing
k_size = 5
drop_p = 0.5

#Turning off random seed to see similar effect for different changes.
# random_seed = 6
torch.backends.cudnn.enabled = False
# torch.manual_seed(None)

In [4]:
#Building the network with 2 convolution layers and following Relu, pooling and dropout layers
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=k_size)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=k_size)
        self.conv2_drop = nn.Dropout2d(drop_p)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
#         x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [5]:
#This function for each epoch performs forward feed and then calculates loss. Then the loss is back propagated to optimize
#the weights of the network. We are printing values for batches for better tracking the progress of our network.
def train(epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = network(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append(
            (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
#             torch.save(network.state_dict(), 'results/model.pth')
#             torch.save(optimizer.state_dict(), 'results/optimizer.pth')

In [6]:
#This function performs testing on the test set. It just compares our prediction for the hand drawn digit to the ground 
#truth, then it returns the correct prediction accuracy for the test set.
def test():
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
        test_loss /= len(test_loader.dataset)
        test_losses.append(test_loss)
        print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
#Variables are changing to see its effects on our neural network
drop_percent = [0.3, 0.4, 0.5, 0.6, 0.7]
batch_size = [40, 80, 100, 120, 150]
kernel_s = [4, 5]

#Iterations counter for tracking progress
iteration = 0

#total iterations 5 x 2 x 5 -> 50
#Training and testing on 5 batch sizes
for i in range(len(batch_size)):
    print("\n\n\n")
    
    batch_size_train = batch_size[i]
    print("Changing the batch size to "+str(batch_size_train))
    
    #training and testing on 2 kernel sizes
    for j in range(len(kernel_s)):
        k_size = kernel_s[j]
        print("Changing the kernel size " + str(k_size))
        
        #training and testing on 5 drop percentages
        for k in range(len(drop_percent)):

            drop_p = drop_percent[k]
            print("Changing the drop percentage - "+ str(drop_p*100) +"%")
            
            
            iteration += 1
            print(str(iteration)+'th iteration -')
            print("Run for Batch size - " + str(batch_size_train) + ", Kernel size - " + str(k_size) + ", Dropout percent - " + str(drop_p))
    
            #Resetting the network after each iteration
            network = Net()
            optimizer = optim.SGD(network.parameters(), lr=learning_rate,
                              momentum=momentum)
            
            #Since we are changing batch sizes, we are loading dataset every time
            train_loader = torch.utils.data.DataLoader(
              torchvision.datasets.MNIST('/files/', train=True, download=True,
                                     transform=torchvision.transforms.Compose([
                                       torchvision.transforms.ToTensor(),
                                       torchvision.transforms.Normalize(
                                         (0.1307,), (0.3081,))
                                     ])),
              batch_size=batch_size_train, shuffle=True)

            test_loader = torch.utils.data.DataLoader(
              torchvision.datasets.MNIST('/files/', train=False, download=True,
                                     transform=torchvision.transforms.Compose([
                                       torchvision.transforms.ToTensor(),
                                       torchvision.transforms.Normalize(
                                         (0.1307,), (0.3081,))
                                     ])),
              batch_size=batch_size_test, shuffle=True)
            
            #Initializing the tracking parameters
            train_losses = []
            train_counter = []
            test_losses = []
            test_counter = [i*len(train_loader.dataset) for i in range(n_epochs + 1)]
            
            #Testing for 1 epochs
            try:
                test()
                for epoch in range(1, n_epochs + 1):
                    train(epoch)
                    test()
            except:
                print('The total number of input tensors is greater than nodes')





Changing the batch size to 40
Changing the kernel size 4
Changing the drop percentage - 30.0%
1th iteration -
Run for Batch size - 40, Kernel size - 4, Dropout percent - 0.3


C:\Users\shree\AppData\Local\Temp\ipykernel_5280\1891330620.py:18: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)



Test set: Avg. loss: 2.3020, Accuracy: 1402/10000 (14%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.333679
Train Epoch: 1 [400/60000 (1%)]	Loss: 2.288738
Train Epoch: 1 [800/60000 (1%)]	Loss: 2.281834
Train Epoch: 1 [1200/60000 (2%)]	Loss: 2.282315
Train Epoch: 1 [1600/60000 (3%)]	Loss: 2.277416
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.248294
Train Epoch: 1 [2400/60000 (4%)]	Loss: 2.227589
Train Epoch: 1 [2800/60000 (5%)]	Loss: 2.182301
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.180198
Train Epoch: 1 [3600/60000 (6%)]	Loss: 2.086556
Train Epoch: 1 [4000/60000 (7%)]	Loss: 1.988017
Train Epoch: 1 [4400/60000 (7%)]	Loss: 1.960136
Train Epoch: 1 [4800/60000 (8%)]	Loss: 1.638998
Train Epoch: 1 [5200/60000 (9%)]	Loss: 1.569332
Train Epoch: 1 [5600/60000 (9%)]	Loss: 1.423344
Train Epoch: 1 [6000/60000 (10%)]	Loss: 1.134210
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.129651
Train Epoch: 1 [6800/60000 (11%)]	Loss: 0.985217
Train Epoch: 1 [7200/60000 (12%)]	Loss: 0.944125
Train Epoch: 1 [7600/60000 (13%

Train Epoch: 1 [4000/60000 (7%)]	Loss: 1.949642
Train Epoch: 1 [4400/60000 (7%)]	Loss: 1.751521
Train Epoch: 1 [4800/60000 (8%)]	Loss: 1.498983
Train Epoch: 1 [5200/60000 (9%)]	Loss: 1.176699
Train Epoch: 1 [5600/60000 (9%)]	Loss: 1.040121
Train Epoch: 1 [6000/60000 (10%)]	Loss: 1.201514
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.059671
Train Epoch: 1 [6800/60000 (11%)]	Loss: 0.723724
Train Epoch: 1 [7200/60000 (12%)]	Loss: 0.966922
Train Epoch: 1 [7600/60000 (13%)]	Loss: 0.939962
Train Epoch: 1 [8000/60000 (13%)]	Loss: 0.513996
Train Epoch: 1 [8400/60000 (14%)]	Loss: 0.650272
Train Epoch: 1 [8800/60000 (15%)]	Loss: 0.526532
Train Epoch: 1 [9200/60000 (15%)]	Loss: 0.542053
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.579033
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.305086
Train Epoch: 1 [10400/60000 (17%)]	Loss: 0.656606
Train Epoch: 1 [10800/60000 (18%)]	Loss: 0.412270
Train Epoch: 1 [11200/60000 (19%)]	Loss: 0.679923
Train Epoch: 1 [11600/60000 (19%)]	Loss: 0.651129
Train Epoch: 1 [1200

Train Epoch: 1 [8400/60000 (14%)]	Loss: 0.797666
Train Epoch: 1 [8800/60000 (15%)]	Loss: 0.812935
Train Epoch: 1 [9200/60000 (15%)]	Loss: 0.834088
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.987138
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.742786
Train Epoch: 1 [10400/60000 (17%)]	Loss: 0.447320
Train Epoch: 1 [10800/60000 (18%)]	Loss: 0.782315
Train Epoch: 1 [11200/60000 (19%)]	Loss: 1.032071
Train Epoch: 1 [11600/60000 (19%)]	Loss: 0.601402
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.786330
Train Epoch: 1 [12400/60000 (21%)]	Loss: 0.813835
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.368225
Train Epoch: 1 [13200/60000 (22%)]	Loss: 0.642412
Train Epoch: 1 [13600/60000 (23%)]	Loss: 0.629916
Train Epoch: 1 [14000/60000 (23%)]	Loss: 0.420121
Train Epoch: 1 [14400/60000 (24%)]	Loss: 0.540048
Train Epoch: 1 [14800/60000 (25%)]	Loss: 0.628882
Train Epoch: 1 [15200/60000 (25%)]	Loss: 1.081580
Train Epoch: 1 [15600/60000 (26%)]	Loss: 0.358896
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.565753
Trai

...

Train Epoch: 1 [30400/60000 (51%)]	Loss: 0.240350
Train Epoch: 1 [31200/60000 (52%)]	Loss: 0.324561
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.319639
Train Epoch: 1 [32800/60000 (55%)]	Loss: 0.294306
Train Epoch: 1 [33600/60000 (56%)]	Loss: 0.468906
Train Epoch: 1 [34400/60000 (57%)]	Loss: 0.270380
Train Epoch: 1 [35200/60000 (59%)]	Loss: 0.407417
Train Epoch: 1 [36000/60000 (60%)]	Loss: 0.268639
Train Epoch: 1 [36800/60000 (61%)]	Loss: 0.369999
Train Epoch: 1 [37600/60000 (63%)]	Loss: 0.250132
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.231158
Train Epoch: 1 [39200/60000 (65%)]	Loss: 0.257950
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.295730
Train Epoch: 1 [40800/60000 (68%)]	Loss: 0.369769
Train Epoch: 1 [41600/60000 (69%)]	Loss: 0.368022
Train Epoch: 1 [42400/60000 (71%)]	Loss: 0.313214
Train Epoch: 1 [43200/60000 (72%)]	Loss: 0.290834
Train Epoch: 1 [44000/60000 (73%)]	Loss: 0.265958
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.197977
Train Epoch: 1 [45600/60000 (76%)]	Loss: 0.327484


Train Epoch: 1 [35200/60000 (59%)]	Loss: 0.439571
Train Epoch: 1 [36000/60000 (60%)]	Loss: 0.303836
Train Epoch: 1 [36800/60000 (61%)]	Loss: 0.467472
Train Epoch: 1 [37600/60000 (63%)]	Loss: 0.262304
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.302565
Train Epoch: 1 [39200/60000 (65%)]	Loss: 0.368399
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.636607
Train Epoch: 1 [40800/60000 (68%)]	Loss: 0.222304
Train Epoch: 1 [41600/60000 (69%)]	Loss: 0.395966
Train Epoch: 1 [42400/60000 (71%)]	Loss: 0.547575
Train Epoch: 1 [43200/60000 (72%)]	Loss: 0.350249
Train Epoch: 1 [44000/60000 (73%)]	Loss: 0.395233
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.655108
Train Epoch: 1 [45600/60000 (76%)]	Loss: 0.277083
Train Epoch: 1 [46400/60000 (77%)]	Loss: 0.484975
Train Epoch: 1 [47200/60000 (79%)]	Loss: 0.229168
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.486933
Train Epoch: 1 [48800/60000 (81%)]	Loss: 0.341350
Train Epoch: 1 [49600/60000 (83%)]	Loss: 0.182936
Train Epoch: 1 [50400/60000 (84%)]	Loss: 0.344833


Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.413716
Train Epoch: 1 [40800/60000 (68%)]	Loss: 0.421539
Train Epoch: 1 [41600/60000 (69%)]	Loss: 0.428006
Train Epoch: 1 [42400/60000 (71%)]	Loss: 0.316097
Train Epoch: 1 [43200/60000 (72%)]	Loss: 0.403365
Train Epoch: 1 [44000/60000 (73%)]	Loss: 0.477935
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.425198
Train Epoch: 1 [45600/60000 (76%)]	Loss: 0.283541
Train Epoch: 1 [46400/60000 (77%)]	Loss: 0.400006
Train Epoch: 1 [47200/60000 (79%)]	Loss: 0.582525
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.582948
Train Epoch: 1 [48800/60000 (81%)]	Loss: 0.236057
Train Epoch: 1 [49600/60000 (83%)]	Loss: 0.332818
Train Epoch: 1 [50400/60000 (84%)]	Loss: 0.350578
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.308076
Train Epoch: 1 [52000/60000 (87%)]	Loss: 0.489230
Train Epoch: 1 [52800/60000 (88%)]	Loss: 0.539413
Train Epoch: 1 [53600/60000 (89%)]	Loss: 0.531692
Train Epoch: 1 [54400/60000 (91%)]	Loss: 0.569761
Train Epoch: 1 [55200/60000 (92%)]	Loss: 0.464950


Train Epoch: 1 [5000/60000 (8%)]	Loss: 2.254218
Train Epoch: 1 [6000/60000 (10%)]	Loss: 2.217825
Train Epoch: 1 [7000/60000 (12%)]	Loss: 2.244808
Train Epoch: 1 [8000/60000 (13%)]	Loss: 2.146240
Train Epoch: 1 [9000/60000 (15%)]	Loss: 2.123428
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.019717
Train Epoch: 1 [11000/60000 (18%)]	Loss: 1.889314
Train Epoch: 1 [12000/60000 (20%)]	Loss: 1.578005
Train Epoch: 1 [13000/60000 (22%)]	Loss: 1.425212
Train Epoch: 1 [14000/60000 (23%)]	Loss: 1.220491
Train Epoch: 1 [15000/60000 (25%)]	Loss: 1.077240
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.979978
Train Epoch: 1 [17000/60000 (28%)]	Loss: 0.864644
Train Epoch: 1 [18000/60000 (30%)]	Loss: 0.917442
Train Epoch: 1 [19000/60000 (32%)]	Loss: 0.624110
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.760718
Train Epoch: 1 [21000/60000 (35%)]	Loss: 0.645691
Train Epoch: 1 [22000/60000 (37%)]	Loss: 0.692645
Train Epoch: 1 [23000/60000 (38%)]	Loss: 0.702828
Train Epoch: 1 [24000/60000 (40%)]	Loss: 0.676060
Train 

Train Epoch: 1 [41000/60000 (68%)]	Loss: 0.550775
Train Epoch: 1 [42000/60000 (70%)]	Loss: 0.639547
Train Epoch: 1 [43000/60000 (72%)]	Loss: 0.621215
Train Epoch: 1 [44000/60000 (73%)]	Loss: 0.575897
Train Epoch: 1 [45000/60000 (75%)]	Loss: 0.550551
Train Epoch: 1 [46000/60000 (77%)]	Loss: 0.476899
Train Epoch: 1 [47000/60000 (78%)]	Loss: 0.574424
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.632919
Train Epoch: 1 [49000/60000 (82%)]	Loss: 0.438338
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.513617
Train Epoch: 1 [51000/60000 (85%)]	Loss: 0.744439
Train Epoch: 1 [52000/60000 (87%)]	Loss: 0.426491
Train Epoch: 1 [53000/60000 (88%)]	Loss: 0.595448
Train Epoch: 1 [54000/60000 (90%)]	Loss: 0.466151
Train Epoch: 1 [55000/60000 (92%)]	Loss: 0.391053
Train Epoch: 1 [56000/60000 (93%)]	Loss: 0.583929
Train Epoch: 1 [57000/60000 (95%)]	Loss: 0.502935
Train Epoch: 1 [58000/60000 (97%)]	Loss: 0.402045
Train Epoch: 1 [59000/60000 (98%)]	Loss: 0.457172

Test set: Avg. loss: 0.2805, Accuracy: 9170/10000